In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
import re

In [2]:
def smote(X,y):
    smote = SMOTE(random_state= 123)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    return X_resampled, y_resampled

In [19]:
def dt_kfold(input_dir, output_path, k_fold_cv, max_depth, is_sample_aug):
    
    pattern = r'filtered_(.*?)\.csv'

    match = re.search(pattern, input_dir)
    if match:
        name_result = match.group(1)
    
    if is_sample_aug:
        input_dir = input_dir
        output_path = output_path

        data = pd.read_csv(input_dir)
        
        smote_result_dict = {}

        smote_result_dict['method'] = 'DT'+'_'+str(int(k_fold_cv))+'_kfold'+'_smote_'+name_result

        skf = StratifiedKFold(n_splits = k_fold_cv, random_state = 123, shuffle = True)

        data_val = data.values
        X = data_val[:,1:]
        y = data_val[:,0]

        smote_dt_model = tree.DecisionTreeClassifier(max_depth = max_depth, random_state = 123)

        smote_metrics_result = np.zeros((k_fold_cv,5))

        i = 0

        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            smote_x, smote_y = smote(X_train,y_train)

            smote_dt_model.fit(smote_x, smote_y)

            smote_y_pred = smote_dt_model.fit(smote_x, smote_y).predict(X_test)

            smote_metrics_result[i,0] = accuracy_score(y_test, smote_y_pred)

            smote_metrics_result[i,1] = matthews_corrcoef(y_test, smote_y_pred)

            smote_tn, smote_fp, smote_fn, smote_tp = confusion_matrix(y_test, smote_y_pred).ravel()

            smote_metrics_result[i,2] = smote_tp / (smote_tp + smote_fn)

            smote_metrics_result[i,3] = smote_tn / (smote_tn + smote_fp)

            smote_y_pred_proba = smote_dt_model.predict_proba(X_test)

            smote_metrics_result[i,4] = roc_auc_score(y_test, smote_y_pred_proba[:,1])

            i += 1

        smote_metrics_results_mean = smote_metrics_result.mean(axis=0)

        smote_result_dict['ACC'] = smote_metrics_results_mean[0]

        smote_result_dict['MCC'] = smote_metrics_results_mean[1]

        smote_result_dict['Sensitivity'] = smote_metrics_results_mean[2]

        smote_result_dict['Specificity'] = smote_metrics_results_mean[3]

        smote_result_dict['AUC'] = smote_metrics_results_mean[4]

        original_data = pd.read_excel(output_path)

        smote_result_df = pd.DataFrame(smote_result_dict, index=[0])

        smote_save_result = pd.concat([original_data, smote_result_df], axis=0)

        smote_save_result.to_excel(output_path, index=False)
            
    else:
        input_dir = input_dir
        output_path = output_path
        data = pd.read_csv(input_dir)
        result_dict = {}
        result_dict['method'] = 'DT'+'_'+str(int(k_fold_cv))+'_kfold_'+name_result

        skf = StratifiedKFold(n_splits = k_fold_cv, random_state = 123, shuffle = True)

        data_val = data.values

        X = data_val[:,1:]
        y = data_val[:,0]

        dt_model = tree.DecisionTreeClassifier(max_depth = max_depth, random_state=123)

        metrics_result = np.zeros((k_fold_cv,5))

        i = 0

        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            y_pred = dt_model.fit(X_train, y_train).predict(X_test)
            metrics_result[i,0] = accuracy_score(y_test, y_pred)
            metrics_result[i,1] = matthews_corrcoef(y_test, y_pred)
            tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
            metrics_result[i,2] = tp / (tp + fn)
            metrics_result[i,3] = tn / (tn + fp)
            y_pred_proba = dt_model.predict_proba(X_test)
            auc = roc_auc_score(y_test, y_pred_proba[:,1])
            metrics_result[i,4] = auc
            i += 1

        metrics_results_mean = metrics_result.mean(axis=0)

        result_dict['ACC'] = metrics_results_mean[0]

        result_dict['MCC'] = metrics_results_mean[1]

        result_dict['Sensitivity'] = metrics_results_mean[2]
        result_dict['Specificity'] = metrics_results_mean[3]


        result_dict['AUC'] = metrics_results_mean[4]

        original_data = pd.read_excel(output_path)
        result_df = pd.DataFrame(result_dict, index=[0])
        save_result = pd.concat([original_data, result_df], axis=0)
        save_result.to_excel(output_path, index=False)

In [20]:
dt_kfold("C:/Users/Li Xiaokang/Desktop/null_importance_result/win31_RF_filtered_0.05_train_PCC_1.csv", 
         "C:/Users/Li Xiaokang/Desktop/win31_kfold_result.xlsx",
         5, 10, 1)

In [3]:
def dt_independent_test(train_path, test_path, output_path, is_sample_aug, max_depth):
    pattern = r'PCC_(.*?)\.csv'

    match = re.search(pattern, test_path)
    if match:
        name_result = match.group(1)
    
    if is_sample_aug:
        output_path = output_path
#         over_result_dict = {}
#         under_result_dict = {}
        smote_result_dict = {}

        smote_result_dict['method'] = 'win31_DT_'+'smote'

        # Split the data set into training and test sets, with stratified sampling based on the 'label' column
        train = pd.read_csv(train_path)
        test = pd.read_csv(test_path)
        train_val = train.values
        test_val = test.values

        X_train = train_val[:,1:]
        X_test = test_val[:,1:]
        y_train = train_val[:,0]
        y_test = test_val[:,0]

#         over_x, over_y = over_sampler(X_train,y_train)
#         under_x, under_y = under_sampler(X_train,y_train)
        smote_x, smote_y = smote(X_train,y_train)


        smote_dt_model = tree.DecisionTreeClassifier(max_depth=max_depth, random_state=123)

        smote_dt_model.fit(smote_x, smote_y)

        smote_y_pred = smote_dt_model.predict(X_test)

        smote_accuracy = accuracy_score(y_test, smote_y_pred)

        smote_result_dict['ACC'] = smote_accuracy

        smote_mcc = matthews_corrcoef(y_test, smote_y_pred)


        smote_result_dict['MCC'] = smote_mcc



        smote_tn, smote_fp, smote_fn, smote_tp = confusion_matrix(y_test, smote_y_pred).ravel()


        smote_sensitivity = smote_tp / (smote_tp + smote_fn)


        smote_specificity = smote_tn / (smote_tn + smote_fp)


        smote_result_dict['Sensitivity'] = smote_sensitivity


        smote_result_dict['Specificity'] = smote_specificity


        smote_y_pred_proba = smote_dt_model.predict_proba(X_test)


        smote_auc = roc_auc_score(y_test, smote_y_pred_proba[:,1])


        smote_result_dict['AUC'] = smote_auc

        original_data = pd.read_excel(output_path)

        smote_result_df = pd.DataFrame(smote_result_dict, index=[0])

        smote_save_result = pd.concat([original_data, smote_result_df], axis=0)

        smote_save_result.to_excel(output_path, index=False)
    else:
        output_path = output_path
        result_dict = {}
        result_dict['method'] = 'DT_imb'

        train = pd.read_csv(train_path)
        test = pd.read_csv(test_path)


        train_val = train.values
        test_val = test.values

        X_train = train_val[:,1:]
        X_test = test_val[:,1:]
        y_train = train_val[:,0]
        y_test = test_val[:,0]

        # Define your random forest model
        dt_model = tree.DecisionTreeClassifier(max_depth=max_depth, random_state=123)

        # Train the model on the entire training set
        dt_model.fit(X_train, y_train)

        # Evaluate the model on the test set
        y_pred = dt_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        result_dict['ACC'] = accuracy

        # calculate the MCC of the model
        mcc = matthews_corrcoef(y_test, y_pred)

        result_dict['MCC'] = mcc

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)

        result_dict['Sensitivity'] = sensitivity
        result_dict['Specificity'] = specificity

        y_pred_proba = dt_model.predict_proba(X_test)
        auc = roc_auc_score(y_test, y_pred_proba[:,1])
        result_dict['AUC'] = auc

        original_data = pd.read_excel(output_path)
        result_df = pd.DataFrame(result_dict, index=[0])
        save_result = pd.concat([original_data, result_df], axis=0)
        save_result.to_excel(output_path, index=False)

In [4]:
dt_independent_test("C:/Users/Li Xiaokang/Desktop/null_importance_result/win31_RF_filtered_0.05_train_PCC_1.csv",
                    "C:/Users/Li Xiaokang/Desktop/null_importance_result/test31_RF_filtered_0.05_null_importances_1.csv",
                    "C:/Users/Li Xiaokang/Desktop/win31_independent_test_result.xlsx" ,
                    1, 10)